# This is the modeling notebook containing step by step process of upload various BERT (Bidirectional Encoder Representations from Transformers) LLMs available on huggingface.  This code comes from Subash Khanal, and this was used to persuade the usage of BERT base uncased for the model that classifies news as fake or real.  I included some transitions to help ease the reading of the code.

# The workflow is as follows:


# 1.) BERT Comparison

# 2.) Optimizing BERT-base-uncased

# Imports

In [ ]:
!pip install datasets

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, log_loss, matthews_corrcoef, confusion_matrix
import pandas as pd
import numpy as np

# Ensure GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# 1.) BERT Comparison

# Load data and split it appropriately. Then convert the pandas dataframe to a hugging face dataset.

In [ ]:
# Load the dataset
from sklearn.model_selection import train_test_split
data = pd.read_csv('Final_Dataset_For_pipeling.csv')

# Combine headline and description for model input
data['text'] = data['Headlines'] + " " + data['description']
data = data[['text', 'Target_final']]
data.rename(columns={"Target_final": "labels"}, inplace=True)

# Split data into train and test sets
train_data, test_data = np.split(data.sample(frac=1, random_state=42), [int(.8*len(data))])

# Convert pandas DataFrames to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# The following hugging face BERTs were selected for testing: distilbert-base-uncased, albert-base-v2, robert-base, bert-base-uncased, microsoft/deberta-v3-small

# Initiate iteration to go over the various models with the following workflow:

1.) Load tokenizer and model

2.) Tokenize text

3.) Convert tokenized text into pytorch ready format

4.) Define the training parameters (learning rate = 2e-5, batch size = 16, number of train epochs = 2, weight decay = 0.01, GPU on)

5.) Train model

6.) Evaluate Model

In [ ]:
# Define models to experiment with
model_names = ["distilbert-base-uncased", "albert-base-v2", "roberta-base", "bert-base-uncased", "microsoft/deberta-v3-small"]

# Dictionary to store results
results = {}

for model_name in model_names:
    print(f"\nEvaluating model: {model_name}")

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

    # Tokenize datasets
    def tokenize_function(example):
        return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)

    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_test = test_dataset.map(tokenize_function, batched=True)

    # Set format for PyTorch
    tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        logging_dir="./logs",
        fp16=True if torch.cuda.is_available() else False,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
    )

    # Train model
    trainer.train()

    # Evaluation metrics
    def compute_metrics(predictions, labels):
        preds = np.argmax(predictions, axis=1)
        accuracy = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
        auc_roc = roc_auc_score(labels, predictions[:, 1])
        logloss = log_loss(labels, predictions[:, 1])
        mcc = matthews_corrcoef(labels, preds)

        # Confusion matrix to get FPR and FNR
        tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
        fp_rate = fp / (fp + tn)
        fn_rate = fn / (fn + tp)

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "auc_roc": auc_roc,
            "log_loss": logloss,
            "mcc": mcc,
            "false_positive_rate": fp_rate,
            "false_negative_rate": fn_rate
        }

    # Evaluate model with progress tracking
    print("Evaluating with progress tracking:")
    predictions = []
    labels = []

    for batch in tqdm(tokenized_test):
        input_ids = batch['input_ids'].unsqueeze(0).to(device)
        attention_mask = batch['attention_mask'].unsqueeze(0).to(device)
        label = batch['labels'].item()

        with torch.no_grad():
            output = model(input_ids, attention_mask=attention_mask)
            logits = output.logits.cpu().numpy()
            pred = torch.argmax(output.logits, dim=1).item()

        predictions.append(logits)
        labels.append(label)

    # Convert predictions to numpy array for metric calculations
    predictions = np.vstack(predictions)

    # Calculate metrics and store results
    metrics = compute_metrics(predictions, labels)
    results[model_name] = metrics

    # Print results
    print(f"Results for {model_name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

# Display final results for model comparison
print("\nFinal Model Comparison:")
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

In [ ]:
# Save model and tokenizer
model.save_pretrained("./best_bert_model")
tokenizer.save_pretrained("./best_bert_model")

In [ ]:
# Compress the model directory
!zip -r best_bert_model.zip best_bert_model

# Download the zipped model file to your local machine
from google.colab import files
files.download("best_bert_model.zip")


# Best model was bert-base-uncased

# 2.) Optimizing BERT-base-uncased

Optimize bert-base-uncased by balancing class weight

In [ ]:
#optimization of bert-base-uncased model

# Calculate class weights (optional, for imbalanced datasets)
from sklearn.utils.class_weight import compute_class_weight

labels = [0, 1]  # Replace with actual labels
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

Training parameters below

In [ ]:
# Define training arguments with optimizations
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    fp16=True,  # Mixed precision for GPU acceleration
    gradient_accumulation_steps=2  # For larger effective batch size
)

# Initiate a weighted trainer that uses the class weights determined above in the loss function (makes the lesser class, the real news, have higher weights)

In [ ]:
# Initialize Trainer with weighted loss (if needed)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Go through the testing sequence again for the weight adjust BERT-base-uncased model

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(val_data)


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define the tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# Apply the tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
from transformers import TrainingArguments

# Define training arguments with consistent save and evaluation strategies
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",         # Set evaluation at the end of each epoch
    save_strategy="epoch",               # Set save strategy to match evaluation strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,                           # Enable mixed-precision training for GPU acceleration
    gradient_accumulation_steps=2,       # Accumulate gradients to simulate larger batch size
    save_total_limit=1,                  # Save only the best model
    load_best_model_at_end=True          # Load the best model at the end
)


In [ ]:
from transformers import Trainer, AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

In [ ]:
from torch.nn.functional import softmax

# Get predictions and true labels
predictions, labels, _ = trainer.predict(eval_dataset)
# Convert logits to class predictions
preds = np.argmax(predictions, axis=1)

In [ ]:
# Accuracy
accuracy = accuracy_score(labels, preds)

# Precision, Recall, and F1 Score
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

# AUC-ROC Score (needs probability scores for the positive class)
auc_roc = roc_auc_score(labels, predictions[:, 1])

# Log Loss (needs probability scores)
logloss = log_loss(labels, softmax(torch.tensor(predictions), dim=1)[:, 1].numpy())

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(labels, preds)

# Confusion Matrix to calculate False Positive and False Negative Rates
tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

# Print each metric
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"Log Loss: {logloss:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"False Positive Rate: {false_positive_rate:.4f}")
print(f"False Negative Rate: {false_negative_rate:.4f}")

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# Get predictions on the validation set
predictions, labels, _ = trainer.predict(eval_dataset)
# Convert logits to probabilities
probs = torch.softmax(torch.tensor(predictions), dim=1)[:, 1].numpy()

# Tune the threshold
best_threshold = 0.5
best_f1 = 0
for threshold in np.arange(0.4, 0.6, 0.01):  # Tuning within a narrow range
    adjusted_preds = (probs > threshold).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, adjusted_preds, average="binary")
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"Optimal Threshold: {best_threshold}, F1 Score: {best_f1:.4f}")

# Fine tune the model with a smaller learning rate (rate of weight change should decrease)

In [ ]:
#Step 2: Fine-Tune with Smaller Learning Rate Decay

# Define new training arguments with a smaller learning rate
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # Smaller learning rate for finer adjustments
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Additional fine-tuning epochs
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True,  # Mixed precision for GPU acceleration
    gradient_accumulation_steps=2,
    load_best_model_at_end=True
)

# Reinitialize Trainer with new training_args
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()

# Add dropout to increase the model's robustness with generalization (randomly removes weights of data in order to reduce overfitting)

In [ ]:
#Step 3: Add Dropout Regularization
from transformers import BertForSequenceClassification

# Load the model with a custom dropout rate
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, hidden_dropout_prob=0.1)

# Reinitialize Trainer with the updated model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model again with dropout
trainer.train()

# Apply focal loss so that the loss is more significant for smaller class

In [ ]:
#Step 4: Implement Focal Loss for Misclassification Handling

import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight

    def forward(self, logits, labels):
        BCE_loss = F.cross_entropy(logits, labels, weight=self.weight, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = ((1 - pt) ** self.gamma) * BCE_loss
        return torch.mean(F_loss)

# Custom Trainer with Focal Loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = FocalLoss(gamma=2)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Use the custom trainer with focal loss
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model with Focal Loss
trainer.train()


# Use cross validation for good practice of checking generalization

In [ ]:
#Step 5: Apply Cross-Validation to Validate Robustness

from sklearn.model_selection import KFold
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define the tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

kfold = KFold(n_splits=5)
results = []

# Cross-Validation loop
for train_index, val_index in kfold.split(data):
    train_data, val_data = data.iloc[train_index], data.iloc[val_index]

    # Convert to Hugging Face Dataset format
    train_dataset = Dataset.from_pandas(train_data)
    eval_dataset = Dataset.from_pandas(val_data)

    # Apply tokenization to each fold
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    eval_dataset = eval_dataset.map(tokenize_function, batched=True)

    # Set format for PyTorch to include input_ids, attention_mask, and labels
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    # Initialize the trainer for each fold
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer
    )

    # Train and evaluate
    trainer.train()
    metrics = trainer.evaluate()
    results.append(metrics)

# Calculate average metrics
avg_metrics = {metric: np.mean([fold[metric] for fold in results]) for metric in results[0]}
print(f"Cross-validated metrics: {avg_metrics}")

# Use early stopping in order to ensure not wasting computers resources if there is no decrease in loss aftter 1 epoch

In [ ]:
#Step 6: Additional Fine-Tuning Epochs with Early Stopping

from transformers import EarlyStoppingCallback

# Define training arguments with early stopping
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True
)

# Initialize Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]  # Stop if no improvement after 1 epoch
)

# Fine-tune the model
trainer.train()

# Evalutate

In [ ]:
# Get predictions and labels
predictions, labels, _ = trainer.predict(eval_dataset)
preds = np.argmax(predictions, axis=1)
probs = softmax(torch.tensor(predictions), dim=1).numpy()

# Accuracy
accuracy = accuracy_score(labels, preds)

# Precision, Recall, F1 Score
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

# AUC-ROC
auc_roc = roc_auc_score(labels, probs[:, 1])

# Log Loss
logloss = log_loss(labels, probs[:, 1])

# MCC (Matthews Correlation Coefficient)
mcc = matthews_corrcoef(labels, preds)

# Confusion Matrix to calculate False Positive and False Negative Rates
tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

# Print each metric
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"Log Loss: {logloss:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"False Positive Rate: {false_positive_rate:.4f}")
print(f"False Negative Rate: {false_negative_rate:.4f}")

# Compare results between the original bert-base-uncased with the tuned version

In [ ]:
#Comparing with the previous fine-tuned models evaluation metrices score
comparison_results = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "AUC-ROC", "Log Loss", "MCC", "False Positive Rate", "False Negative Rate"],
    "Initial Model": [0.8031, 0.7696, 0.8087, 0.7887, 0.8880, 0.4491, 0.6051, 0.2017,0.1913],  # Replace with initial values
    "Optimized Model": [accuracy, precision, recall, f1, auc_roc, logloss, mcc, false_positive_rate, false_negative_rate]
}

import pandas as pd
comparison_df = pd.DataFrame(comparison_results)
print(comparison_df)


# save the model's architecture

In [ ]:
# Save the optimized model and tokenizer in a separate directory
model.save_pretrained("./optimized_final_bert_model")
tokenizer.save_pretrained("./optimized_final_bert_model")


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load the optimized model
model = AutoModelForSequenceClassification.from_pretrained("./optimized_final_bert_model")


# Apply a quantization to the model in order to reduce the amount of computation power it consumes when running

In [ ]:
import torch

# Apply dynamic quantization to reduce model size and make it faster on CPU
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [ ]:
# Save the quantized model's state dictionary
torch.save(quantized_model.state_dict(), "./quantized_final_bert_model.pth")


In [ ]:
# Load the optimized model structure
base_model = AutoModelForSequenceClassification.from_pretrained("./optimized_final_bert_model")

# Apply dynamic quantization to match the saved quantized model structure
quantized_model = torch.quantization.quantize_dynamic(
    base_model, {torch.nn.Linear}, dtype=torch.qint8
)

# Load the saved quantized weights
quantized_model.load_state_dict(torch.load("./quantized_final_bert_model.pth"))

# Set the model to evaluation mode
quantized_model.eval()


# evaluate quantized version of model

In [ ]:
 # Calculate metrics of quantized model

def evaluate_model(model, eval_dataset):
    # Get predictions
    predictions, labels, _ = trainer.predict(eval_dataset)
    preds = np.argmax(predictions, axis=1)
    probs = softmax(torch.tensor(predictions), dim=1).numpy()

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    auc_roc = roc_auc_score(labels, probs[:, 1])
    logloss = log_loss(labels, probs[:, 1])
    mcc = matthews_corrcoef(labels, preds)

    # Confusion matrix for FPR and FNR
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

    metrics = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "AUC-ROC": auc_roc,
        "Log Loss": logloss,
        "MCC": mcc,
        "False Positive Rate": false_positive_rate,
        "False Negative Rate": false_negative_rate
    }
    return metrics


In [ ]:
# putting quantized model is in evaluation mode
quantized_model.eval()

# Evaluate and print metrics
quantized_metrics = evaluate_model(quantized_model, eval_dataset)
print("Quantized Model Metrics:", quantized_metrics)


In [ ]:
# Optimized model metrics for reference
optimized_metrics = {
    "Accuracy": 0.9588,
    "Precision": 0.9416,
    "Recall": 0.9971,
    "F1 Score": 0.9685,
    "AUC-ROC": 0.9942,
    "Log Loss": 0.2073,
    "MCC": 0.9123,
    "False Positive Rate": 0.1079,
    "False Negative Rate": 0.0029
}

# Print comparison
print("\nComparison of Optimized and Quantized Model Metrics:")
for metric, opt_value in optimized_metrics.items():
    quant_value = quantized_metrics[metric]
    print(f"{metric}: Optimized = {opt_value:.4f}, Quantized = {quant_value:.4f}")


# save quantized model

In [ ]:
# Save the quantized model's state dictionary
torch.save(quantized_model.state_dict(), "quantized_final_bert_model.pth")

In [ ]:
import zipfile

# Create a zip file and add the model file to it
with zipfile.ZipFile("quantized_final_bert_model.zip", "w") as zipf:
    zipf.write("quantized_final_bert_model.pth")

In [ ]:
from google.colab import files
files.download("quantized_final_bert_model.zip")


In [ ]:
# Check if the zip file was created successfully
!zipinfo quantized_final_bert_model.zip


In [ ]:
from google.colab import files

# Download the zip file
files.download("quantized_final_bert_model.zip")

In [ ]:
import zipfile

# Unzip the file
with zipfile.ZipFile("quantized_final_bert_model.zip", "r") as zipf:
    zipf.extractall()  # Extracts `quantized_final_bert_model.pth`

# 3.) Final model code to go into Streamlit APp

# Lines of code to go into the Streamlit Application

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

# Load the model structure
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    base_model, {torch.nn.Linear}, dtype=torch.qint8
)

# Load the quantized weights
quantized_model.load_state_dict(torch.load("quantized_final_bert_model.pth"))

# Set the model to evaluation mode
quantized_model.eval()


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
from torch.nn.functional import softmax

def predict_news(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = quantized_model(**inputs)
        logits = outputs.logits
        # Calculate confidence score
        confidence = softmax(logits, dim=1).max().item() * 100  # Confidence in percentage
        # Determine prediction label
        prediction = "real" if logits.argmax() == 1 else "fake"

    return prediction, confidence

In [ ]:
# Get input from user
text_input = input("Enter a news headline or description: ")

# Get the prediction and confidence score
prediction, confidence = predict_news(text_input)

# Display the results
print(f"Prediction: {prediction}")
print(f"Confidence: {confidence:.2f}%")


The BERT-base-uncased was chosen as the best LLM to classify news as real or fake.  The model underwent several rounds of fine tuning in order to increase performance metrics as much as possible despite class imbalance issue causing generalization issues.  The model is ready for deployment onto Streamlit.